<a href="https://colab.research.google.com/github/eduard0buarque/datascience/blob/main/orcamento/01_Engenharia_analise_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from datetime import date

%matplotlib inline

drive.mount('/content/drive')
caminho = '/content/drive/My Drive/Datascience/orcamento/dataset/'
dataset_final = caminho + 'tratado/'

MessageError: ignored

In [ ]:
def le_planilha(planilha_dados, separador = ",", decimais = ",", encode='utf-8'):
  df_dados = pd.read_csv(caminho + planilha_dados, decimal=decimais, sep=separador, encoding=encode)
  return df_dados

In [ ]:
def cria_planilha(dataframe, nome, caminho):
  dataframe.to_csv(caminho + nome)

In [ ]:
def calculaIdade(birthDate): 
    hoje = date.today() 
    idade = hoje.year - birthDate.year - ((hoje.month, hoje.day) < (birthDate.month, birthDate.day)) 
  
    return idade 

##Atendimentos - Urgencia e Internacao

In [ ]:
df_atendimentos = le_planilha('atendimentos.csv', separador=";")
df_atendimentos.head()

,CD_PACIENTE,CD_MULTI_EMPRESA,CD_ATENDIMENTO_INT,DH_ATENDIMENTO_INT,CD_CONVENIO_INT,DH_ALTA_INT,CD_ATENDIMENTO_URG,DH_ATENDIMENTO_URG,CD_CONVENIO_URG,DH_ALTA_URG,TP_ATENDIMENTO
0,3,7,3912301,08/06/2022 17:53:30,53,09/06/2022 15:00:00,3910658,08/06/2022 12:18:17,53,08/06/2022 17:50:49,I
1,1071,7,3462858,09/01/2022 22:58:17,5,15/01/2022 20:50:45,3462758,09/01/2022 20:47:51,5,09/01/2022 22:57:08,I
2,1520,7,4401673,14/11/2022 15:09:00,5,16/11/2022 17:07:54,4399386,14/11/2022 08:07:40,5,14/11/2022 14:59:00,I
3,3067,7,3523351,27/01/2022 21:34:00,48,30/01/2022 16:26:05,3522814,27/01/2022 16:32:54,48,27/01/2022 21:32:37,I
4,3935,7,4071446,29/07/2022 20:50:51,6,04/08/2022 14:20:00,4071254,29/07/2022 17:56:39,6,29/07/2022 20:46:45,I


Avaliando Tipos de Dados e Valores Nulos:

In [ ]:
df_atendimentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12124 entries, 0 to 12123
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CD_PACIENTE         12124 non-null  int64 
 1   CD_MULTI_EMPRESA    12124 non-null  int64 
 2   CD_ATENDIMENTO_INT  12124 non-null  int64 
 3   DH_ATENDIMENTO_INT  12124 non-null  object
 4   CD_CONVENIO_INT     12124 non-null  int64 
 5   DH_ALTA_INT         12124 non-null  object
 6   CD_ATENDIMENTO_URG  12124 non-null  int64 
 7   DH_ATENDIMENTO_URG  12124 non-null  object
 8   CD_CONVENIO_URG     12124 non-null  int64 
 9   DH_ALTA_URG         12124 non-null  object
 10  TP_ATENDIMENTO      12124 non-null  object
dtypes: int64(6), object(5)
memory usage: 1.0+ MB


In [ ]:
df_atendimentos.isnull().sum()

CD_PACIENTE           0
CD_MULTI_EMPRESA      0
CD_ATENDIMENTO_INT    0
DH_ATENDIMENTO_INT    0
CD_CONVENIO_INT       0
DH_ALTA_INT           0
CD_ATENDIMENTO_URG    0
DH_ATENDIMENTO_URG    0
CD_CONVENIO_URG       0
DH_ALTA_URG           0
TP_ATENDIMENTO        0
dtype: int64

Convertendo campos Data/Hora de Object para DateTime e revendo a estrutura

In [ ]:
df_atendimentos['DH_ATENDIMENTO_INT'] = pd.to_datetime(df_atendimentos['DH_ATENDIMENTO_INT'], dayfirst=True)
df_atendimentos['DH_ALTA_INT'] = pd.to_datetime(df_atendimentos['DH_ALTA_INT'], dayfirst=True)
df_atendimentos['DH_ATENDIMENTO_URG'] = pd.to_datetime(df_atendimentos['DH_ATENDIMENTO_URG'], dayfirst=True)
df_atendimentos['DH_ALTA_URG'] = pd.to_datetime(df_atendimentos['DH_ALTA_URG'], dayfirst=True)

In [ ]:
df_atendimentos['DH_ATENDIMENTO_URG'].min()

Timestamp('2022-01-02 00:55:23')

In [ ]:
df_atendimentos.drop('TP_ATENDIMENTO', inplace=True, axis=1)
df_atendimentos.drop('CD_MULTI_EMPRESA', inplace=True, axis=1)

In [ ]:
cria_planilha(df_atendimentos, 'atendimento.csv', dataset_final)

##Contas de Internacao (alvo da predicao)

In [ ]:
df_conta_int = le_planilha('faturamento.csv', separador=";")

In [ ]:
df_conta_int.TP_CONTA.unique()

array(['Hospitalar'], dtype=object)

In [ ]:
colunas_conta = ['CD_ATENDIMENTO',	'CD_PRESTADOR', 'CD_CONVENIO',	'CD_PRO_INT_PROCEDIMENTO_ENTRAD', 'VL_TOTAL_CONTA']

In [ ]:
df_contas_temp = df_conta_int[colunas_conta]

In [ ]:
df_contas_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23178 entries, 0 to 23177
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   CD_ATENDIMENTO                  23178 non-null  int64  
 1   CD_PRESTADOR                    23178 non-null  int64  
 2   CD_CONVENIO                     23178 non-null  int64  
 3   CD_PRO_INT_PROCEDIMENTO_ENTRAD  23178 non-null  int64  
 4   VL_TOTAL_CONTA                  23178 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 905.5 KB


In [ ]:
cria_planilha(df_contas_temp, 'contas_internacao.csv', dataset_final)

##Conta Urgência

In [ ]:
df_conta_urg = le_planilha('faturamento_urg.csv', separador=";")  
df_conta_urg.head()

,CD_ATENDIMENTO,DT_ATENDIMENTO,DT_ALTA,CD_PRESTADOR,CD_CONTA_FFCV,TP_CONTA,CD_CONVENIO,CD_PRO_INT_PROCEDIMENTO_ENTRAD,VL_TOTAL_CONTA
0,3492787,18/01/2022 00:00:00,18/01/2022 16:10:44,31026,2907457,Ambulatorial,48,10101039.0,648.27
1,3474848,12/01/2022 00:00:00,12/01/2022 23:38:17,7010,2892310,Ambulatorial,742,10101039.0,0.01
2,3507533,22/01/2022 00:00:00,23/01/2022 02:37:25,35430,2920176,Ambulatorial,5,10101039.0,1755.64
3,3484247,15/01/2022 00:00:00,15/01/2022 19:51:28,7010,2900292,Ambulatorial,13,97000001.0,1662.91
4,3486131,17/01/2022 00:00:00,17/01/2022 09:28:38,26646,2902030,Ambulatorial,7,10101039.0,2102.84


In [ ]:
colunas_conta_urg = ['CD_ATENDIMENTO']
df_total_conta = df_conta_urg.groupby(colunas_conta_urg, as_index=False)['VL_TOTAL_CONTA'].sum()

In [ ]:
df_total_conta.tail()

,CD_ATENDIMENTO,VL_TOTAL_CONTA
12236,4473328,1757.08
12237,4473341,1749.37
12238,4473346,1200.94
12239,4473370,1428.42
12240,4473379,1425.41


In [ ]:
cria_planilha(df_total_conta, 'total_conta_urgencia.csv', dataset_final)

##Pedidos de Exames Laboratorio e Imagem

In [ ]:
df_exa_lab = le_planilha('exame_lab.csv', separador=";")
df_exa_imagem = le_planilha('exame_imagem.csv', separador=";")

In [ ]:
df_exa_lab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162461 entries, 0 to 162460
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   CD_PACIENTE       162461 non-null  int64  
 1   NM_PACIENTE       162461 non-null  object 
 2   CD_ATENDIMENTO    162461 non-null  int64  
 3   CD_PED_LAB        162461 non-null  int64  
 4   CD_SETOR          162461 non-null  int64  
 5   CD_EXA_LAB        162461 non-null  int64  
 6   NM_EXA_LAB        162461 non-null  object 
 7   NM_MNEMONICO      162461 non-null  object 
 8   CD_PRESTADOR      162449 non-null  float64
 9   NM_PRESTADOR      162378 non-null  object 
 10  CD_CONVENIO       162461 non-null  int64  
 11  NM_CONVENIO       162461 non-null  object 
 12  CD_CON_PLA        162461 non-null  int64  
 13  CD_PRE_MED        162081 non-null  float64
 14  CD_MULTI_EMPRESA  162461 non-null  int64  
 15  TP_SOLICITACAO    162461 non-null  object 
 16  DT_COLETA         16

In [ ]:
df_exa_lab['DT_COLETA'] = pd.to_datetime(df_exa_lab['DT_COLETA'], dayfirst=True)
df_exa_lab['HR_COLETA'] = pd.to_datetime(df_exa_lab['HR_COLETA'], dayfirst=True)

In [ ]:
df_exa_imagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18133 entries, 0 to 18132
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   PEDIDO                         18133 non-null  int64  
 1   DT_PEDIDO                      18133 non-null  object 
 2   HR_PEDIDO                      18133 non-null  object 
 3   CD_ATENDIMENTO                 18133 non-null  int64  
 4   CD_CONVENIO                    18133 non-null  int64  
 5   NM_CONVENIO                    18133 non-null  object 
 6   TP_CONVENIO                    18133 non-null  object 
 7   PLANO_CONVENIO                 18133 non-null  int64  
 8   PRESTADOR                      18132 non-null  float64
 9   FLAG_MOTIVO_DA_SOLICITACAO     18133 non-null  object 
 10  TIPO_DO_MOTIVO_DA_SOLICITACAO  18133 non-null  object 
 11  CD_PRESCRICAO                  18132 non-null  float64
 12  EXAME                          18133 non-null 

In [ ]:
df_exa_imagem['DT_PEDIDO'] = pd.to_datetime(df_exa_imagem['DT_PEDIDO'], dayfirst=True)
df_exa_imagem['HR_PEDIDO'] = pd.to_datetime(df_exa_imagem['HR_PEDIDO'], dayfirst=True)

In [ ]:
df_exa_lab.isnull().sum()

CD_PACIENTE           0
NM_PACIENTE           0
CD_ATENDIMENTO        0
CD_PED_LAB            0
CD_SETOR              0
CD_EXA_LAB            0
NM_EXA_LAB            0
NM_MNEMONICO          0
CD_PRESTADOR         12
NM_PRESTADOR         83
CD_CONVENIO           0
NM_CONVENIO           0
CD_CON_PLA            0
CD_PRE_MED          380
CD_MULTI_EMPRESA      0
TP_SOLICITACAO        0
DT_COLETA             0
HR_COLETA             0
dtype: int64

In [ ]:
df_exa_lab.drop(['CD_PRESTADOR', 'NM_PRESTADOR', 'CD_CON_PLA','CD_PRE_MED','CD_MULTI_EMPRESA'], axis=1, inplace=True)

In [ ]:
df_exa_lab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162461 entries, 0 to 162460
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   CD_PACIENTE     162461 non-null  int64         
 1   NM_PACIENTE     162461 non-null  object        
 2   CD_ATENDIMENTO  162461 non-null  int64         
 3   CD_PED_LAB      162461 non-null  int64         
 4   CD_SETOR        162461 non-null  int64         
 5   CD_EXA_LAB      162461 non-null  int64         
 6   NM_EXA_LAB      162461 non-null  object        
 7   NM_MNEMONICO    162461 non-null  object        
 8   CD_CONVENIO     162461 non-null  int64         
 9   NM_CONVENIO     162461 non-null  object        
 10  TP_SOLICITACAO  162461 non-null  object        
 11  DT_COLETA       162461 non-null  datetime64[ns]
 12  HR_COLETA       162461 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(6), object(5)
memory usage: 16.1+ MB


In [ ]:
df_exa_imagem.isnull().sum()

PEDIDO                               0
DT_PEDIDO                            0
HR_PEDIDO                            0
CD_ATENDIMENTO                       0
CD_CONVENIO                          0
NM_CONVENIO                          0
TP_CONVENIO                          0
PLANO_CONVENIO                       0
PRESTADOR                            1
FLAG_MOTIVO_DA_SOLICITACAO           0
TIPO_DO_MOTIVO_DA_SOLICITACAO        0
CD_PRESCRICAO                        1
EXAME                                0
DESCRICAO_DO_EXAME                   0
DESCRICAO_DO_MNEMONICO           10305
FLAG_REALIZADO                       0
CHAVE_DO_ITEM_PRESCRITO              1
dtype: int64

In [ ]:
df_exa_imagem.drop(['PLANO_CONVENIO','PRESTADOR','CD_PRESCRICAO','DESCRICAO_DO_MNEMONICO','CHAVE_DO_ITEM_PRESCRITO'], inplace=True, axis=1)

In [ ]:
df_exa_imagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18133 entries, 0 to 18132
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   PEDIDO                         18133 non-null  int64         
 1   DT_PEDIDO                      18133 non-null  datetime64[ns]
 2   HR_PEDIDO                      18133 non-null  datetime64[ns]
 3   CD_ATENDIMENTO                 18133 non-null  int64         
 4   CD_CONVENIO                    18133 non-null  int64         
 5   NM_CONVENIO                    18133 non-null  object        
 6   TP_CONVENIO                    18133 non-null  object        
 7   FLAG_MOTIVO_DA_SOLICITACAO     18133 non-null  object        
 8   TIPO_DO_MOTIVO_DA_SOLICITACAO  18133 non-null  object        
 9   EXAME                          18133 non-null  int64         
 10  DESCRICAO_DO_EXAME             18133 non-null  object        
 11  FLAG_REALIZADO 

In [ ]:
cria_planilha(df_exa_imagem, 'exames_imagem.csv', dataset_final)
cria_planilha(df_exa_lab, 'exames_laboratorio.csv', dataset_final)

##Documentos Eletronicos

In [ ]:
df_doc_eletronico = le_planilha('doc_eletronicos.csv')

In [ ]:
df_doc_eletronico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94316 entries, 0 to 94315
Data columns (total 1 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   CD_DOCUMENTO;NM_DOCUMENTO;CD_ATENDIMENTO  94316 non-null  object
dtypes: object(1)
memory usage: 737.0+ KB


In [ ]:
cria_planilha(df_doc_eletronico, 'doc_eletronico.csv', dataset_final)

##Pacientes

In [ ]:
df_pacientes = le_planilha('pacientes.csv', separador=";")

In [ ]:
df_pacientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9613 entries, 0 to 9612
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CD_PACIENTE      9613 non-null   int64  
 1   DT_NASCIMENTO    9613 non-null   object 
 2   TP_SEXO          9613 non-null   object 
 3   TP_ESTADO_CIVIL  9613 non-null   object 
 4   CD_CIDADE        9613 non-null   int64  
 5   NR_CEP           9613 non-null   int64  
 6   CD_NATURALIDADE  3661 non-null   float64
dtypes: float64(1), int64(3), object(3)
memory usage: 525.8+ KB


In [ ]:
df_pacientes['DT_NASCIMENTO'] = pd.to_datetime(df_pacientes['DT_NASCIMENTO'], dayfirst=True)

In [ ]:
df_pacientes['CD_CIDADE'].unique().shape

(200,)

In [ ]:
df_pacientes['CD_NATURALIDADE'].unique().shape

(260,)

In [ ]:
df_pacientes['NR_CEP'].unique().shape

(5824,)

In [ ]:
df_pacientes.drop(['NR_CEP', 'CD_NATURALIDADE'], axis=1, inplace=True)

In [ ]:
df_pacientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9613 entries, 0 to 9612
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CD_PACIENTE      9613 non-null   int64         
 1   DT_NASCIMENTO    9613 non-null   datetime64[ns]
 2   TP_SEXO          9613 non-null   object        
 3   TP_ESTADO_CIVIL  9613 non-null   object        
 4   CD_CIDADE        9613 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 375.6+ KB


In [ ]:
df_pacientes['IDADE'] = df_pacientes['DT_NASCIMENTO'].apply(calculaIdade)

In [ ]:
df_pacientes.head()

,CD_PACIENTE,DT_NASCIMENTO,TP_SEXO,TP_ESTADO_CIVIL,CD_CIDADE,IDADE
0,3,2008-03-31 02:00:00,M,C,22651,14
1,1071,1954-05-10 02:00:00,F,C,22571,68
2,1520,1955-04-01 02:00:00,F,C,22651,67
3,3067,1938-02-05 02:00:00,F,S,22651,84
4,3935,1951-12-01 02:00:00,F,C,22651,71


In [ ]:
cria_planilha(df_pacientes, 'pacientes.csv', dataset_final)

##Prescrição

In [ ]:
df_prescricao = le_planilha('Prescricao.csv',separador=";", encode="latin1")

<ipython-input-45-30119b8d6ebe>:1: DtypeWarning: Columns (29,31,33,35,36,37,39,41) have mixed types.Specify dtype option on import or set low_memory=False.
  df_prescricao = le_planilha('Prescricao.csv',separador=";", encode="latin1")


In [ ]:
df_prescricao.head()

,ID_PRESCRICAO,CODIGO_ATENDIMENTO,CODIGO_PACIENTE,NOME_PACIENTE,DATA_DE_REFERENCIA,TIPO_DE_PRESCRICAO,DESCRICAO_DO_TIPO_PRESCRICAO,PRESTADOR_DA_PRESCRICAO,NOME_DO_PRESTADOR,TIPO_DO_ITEM,...,CD_PRO_FAT_EXA_LAB,DS_PRO_FAT_EXA_LAB,CD_TIPO_DIETA,DS_TIPO_DIETA,CD_SANGUE_DERIVADOS,DS_SANGUE_DERIVADOS,CD_PRO_FAT_SANGUE,DS_PRO_FAT_SANGUE,CD_PRODUTO,DS_PRODUTO
0,12012591,4428389,3177869,ROSELI JANCZUR VERONEZI,22/11/2022 00:00:00,M,Medico,29647,CAROLINA PINTO NOGUEIRA,Item,...,40304906.0,DIMERO D - PESQUISA E/OU DOSAGEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12012591,4428389,3177869,ROSELI JANCZUR VERONEZI,22/11/2022 00:00:00,M,Medico,29647,CAROLINA PINTO NOGUEIRA,Item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12012591,4428389,3177869,ROSELI JANCZUR VERONEZI,22/11/2022 00:00:00,M,Medico,29647,CAROLINA PINTO NOGUEIRA,Item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9314748,3443144,3224655,FRANCISCO AGNALDO DANNI COSTA PINTO,03/01/2022 00:00:00,M,Medico,29109,HEITOR ANDREI MIRANDA DE CARVALHO,Item,...,40302580.0,UREIA PESQUISA E OU DOSAGEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9314583,3442871,718773,SAMUEL KAZUO CANCIAN,03/01/2022 00:00:00,M,Medico,29109,HEITOR ANDREI MIRANDA DE CARVALHO,Item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012038.0,DIPIFARMA 500MG/ML AMP 2ML


In [ ]:
df_prescricao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316815 entries, 0 to 316814
Data columns (total 44 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   ID_PRESCRICAO                 316815 non-null  int64  
 1   CODIGO_ATENDIMENTO            316815 non-null  int64  
 2   CODIGO_PACIENTE               316815 non-null  int64  
 3   NOME_PACIENTE                 316815 non-null  object 
 4   DATA_DE_REFERENCIA            316815 non-null  object 
 5   TIPO_DE_PRESCRICAO            316815 non-null  object 
 6   DESCRICAO_DO_TIPO_PRESCRICAO  316815 non-null  object 
 7   PRESTADOR_DA_PRESCRICAO       316815 non-null  int64  
 8   NOME_DO_PRESTADOR             316815 non-null  object 
 9   TIPO_DO_ITEM                  316815 non-null  object 
 10  DATA_HORA_PRESCRICAO          316815 non-null  object 
 11  ID_ITEM_PRESCRITO             316815 non-null  int64  
 12  ESQUEMA                       316815 non-nul

In [ ]:
colunas_prescricao = ['ID_ITEM_PRESCRITO','ESQUEMA','DESCRICAO_DO_ESQUEMA','ITEM_DE_PRESCRICAO','NOME_ITEM_PRESCRICAO','FORMA_APLICACAO','NOME_FORMA_APLICACAO','FREQUENCIA','DESCRICAO_DA_FREQUENCIA','QUANTIDADE_PRESCRITA','SE_NECESSARIO','UNIDADE_DO_PRODUTO','DESCRICAO_UNIDADE_DO_PRODUTO','DATA_HORA_ITEM','DIAS_PARA_APLICACAO','CODIGO_PROCEDIMENTO','DESCRICAO_PROCEDIMENTO','CD_DIAG_IMAGEM','DS_DIAG_IMAGEM','CD_PRO_FAT_EXA_RX','DS_PRO_FAT_EXA_RX','CD_SADT','NM_SADT','CD_PRO_FAT_EXA_LAB','DS_PRO_FAT_EXA_LAB','CD_TIPO_DIETA','DS_TIPO_DIETA','CD_SANGUE_DERIVADOS','DS_SANGUE_DERIVADOS','CD_PRO_FAT_SANGUE','DS_PRO_FAT_SANGUE','CD_PRODUTO','DS_PRODUTO']

In [ ]:
print('Dietas Nulas:',  df_prescricao.shape[0]-df_prescricao['CD_TIPO_DIETA'].isnull().sum())

Dietas Nulas: 5712


In [ ]:
df_dietas = df_prescricao[df_prescricao['CD_TIPO_DIETA'].isnull() == False]

In [ ]:
df_dietas['CD_TIPO_DIETA'].unique()

array(['G001', 'G091', 'B001', 'JEJUM', 'G87', 'B003', 'LIQ069', 'G003',
       'LEV001', 'B004', 'B097', 'G009', 'G085', 'SNE071', 'LIQ086',
       'A001', 'LIQ084', 'SNE033', 'SNE81', 'LEV004', 'P116', 'P001',
       'G098', 'G094', 'G086', 'L001', 'A021', 'B104', 'A035', 'G099',
       'H003', 'B096', 'LEV003', 'A030', 'G005', 'NP030', 'L003', 'G071',
       'LEV031', 'P106', 'SNE058', 'A012', 'LEV005', 'A018', 'B106',
       'LIQ002', 'A019', 'A003', 'NP012', 'A011', 'G088', 'L005',
       'SNE057', 'P004', 'B092', 'A020', 'P110', 'A032', 'G097', 'L004',
       'B074', 'LIQ085', 'A008', 'LEV011', 'SNE059', 'B009', 'A042',
       'LEV027', 'B098', 'B007', 'G013', 'A036', 'P002', 'G007', 'SNE077',
       'B091', 'P014', 'B094', 'G092', 'A027', 'B002', 'P003', 'B103',
       'LEV009', 'A002', 'B099', 'LEV002', 'P005', 'B005', 'A031', 'G014',
       'SNE070', 'G089', 'SNE098', 'G100', 'P113', 'A007', 'A015', 'L110',
       'A014', 'LEV012', 'L106', 'LEV024', 'B008', 'SNE114', 'LEV018',

In [ ]:
colunas_dieta = ['ID_PRESCRICAO',	'CODIGO_ATENDIMENTO',	'CODIGO_PACIENTE',	'NOME_PACIENTE', 'DATA_DE_REFERENCIA',	'TIPO_DE_PRESCRICAO',	'DESCRICAO_DO_TIPO_PRESCRICAO',	'PRESTADOR_DA_PRESCRICAO',	'NOME_DO_PRESTADOR',	'TIPO_DO_ITEM',	'CD_TIPO_DIETA', 'DS_TIPO_DIETA']

In [ ]:
cria_planilha(df_dietas[colunas_dieta], 'dietas_prescricao.csv', dataset_final)

In [ ]:
df_exames = df_prescricao[(df_prescricao.CD_PRO_FAT_EXA_LAB.isnull() == False) | (df_prescricao.CD_PRO_FAT_EXA_RX.isnull() == False)]

In [ ]:
df_exames.head(4)

,ID_PRESCRICAO,CODIGO_ATENDIMENTO,CODIGO_PACIENTE,NOME_PACIENTE,DATA_DE_REFERENCIA,TIPO_DE_PRESCRICAO,DESCRICAO_DO_TIPO_PRESCRICAO,PRESTADOR_DA_PRESCRICAO,NOME_DO_PRESTADOR,TIPO_DO_ITEM,...,CD_PRO_FAT_EXA_LAB,DS_PRO_FAT_EXA_LAB,CD_TIPO_DIETA,DS_TIPO_DIETA,CD_SANGUE_DERIVADOS,DS_SANGUE_DERIVADOS,CD_PRO_FAT_SANGUE,DS_PRO_FAT_SANGUE,CD_PRODUTO,DS_PRODUTO
0,12012591,4428389,3177869,ROSELI JANCZUR VERONEZI,22/11/2022 00:00:00,M,Medico,29647,CAROLINA PINTO NOGUEIRA,Item,...,40304906.0,DIMERO D - PESQUISA E/OU DOSAGEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12012591,4428389,3177869,ROSELI JANCZUR VERONEZI,22/11/2022 00:00:00,M,Medico,29647,CAROLINA PINTO NOGUEIRA,Item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12012591,4428389,3177869,ROSELI JANCZUR VERONEZI,22/11/2022 00:00:00,M,Medico,29647,CAROLINA PINTO NOGUEIRA,Item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9314748,3443144,3224655,FRANCISCO AGNALDO DANNI COSTA PINTO,03/01/2022 00:00:00,M,Medico,29109,HEITOR ANDREI MIRANDA DE CARVALHO,Item,...,40302580.0,UREIA PESQUISA E OU DOSAGEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
exames_prescricao = ['ID_ITEM_PRESCRITO','ESQUEMA','DESCRICAO_DO_ESQUEMA','ITEM_DE_PRESCRICAO','NOME_ITEM_PRESCRICAO','FREQUENCIA','DESCRICAO_DA_FREQUENCIA','QUANTIDADE_PRESCRITA','DATA_HORA_ITEM','DIAS_PARA_APLICACAO','CODIGO_PROCEDIMENTO','DESCRICAO_PROCEDIMENTO','CD_DIAG_IMAGEM','DS_DIAG_IMAGEM','CD_PRO_FAT_EXA_RX','DS_PRO_FAT_EXA_RX','CD_SADT','NM_SADT','CD_PRO_FAT_EXA_LAB','DS_PRO_FAT_EXA_LAB']
df_prescricao_exames = df_exames[exames_prescricao]
cria_planilha(df_prescricao_exames, 'exames_prescricao.csv', dataset_final)

In [ ]:
df_prescricao[df_prescricao.DS_DIAG_IMAGEM.isnull()==False].shape

(18052, 44)

In [ ]:
df_prescricao_produto = df_prescricao[df_prescricao.CD_PRODUTO.isnull()==False]

In [ ]:
df_prescricao_produto.head()

,ID_PRESCRICAO,CODIGO_ATENDIMENTO,CODIGO_PACIENTE,NOME_PACIENTE,DATA_DE_REFERENCIA,TIPO_DE_PRESCRICAO,DESCRICAO_DO_TIPO_PRESCRICAO,PRESTADOR_DA_PRESCRICAO,NOME_DO_PRESTADOR,TIPO_DO_ITEM,...,CD_PRO_FAT_EXA_LAB,DS_PRO_FAT_EXA_LAB,CD_TIPO_DIETA,DS_TIPO_DIETA,CD_SANGUE_DERIVADOS,DS_SANGUE_DERIVADOS,CD_PRO_FAT_SANGUE,DS_PRO_FAT_SANGUE,CD_PRODUTO,DS_PRODUTO
4,9314583,3442871,718773,SAMUEL KAZUO CANCIAN,03/01/2022 00:00:00,M,Medico,29109,HEITOR ANDREI MIRANDA DE CARVALHO,Item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012038.0,DIPIFARMA 500MG/ML AMP 2ML
5,9314583,3442871,718773,SAMUEL KAZUO CANCIAN,03/01/2022 00:00:00,M,Medico,29109,HEITOR ANDREI MIRANDA DE CARVALHO,Item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009387.0,CLORIDRATO DE ONDANSETRONA 2MG/ML AMP 2ML
8,9307297,3440279,1144975,VIVIANE DE LIMA CARPEGIANI,02/01/2022 00:00:00,M,Medico,2506,MICHELI MIKAELI COSTA DA PONTE SOUZA,Item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005641.0,CEFTRIAXONA 1G FA IV
9,9307297,3440279,1144975,VIVIANE DE LIMA CARPEGIANI,02/01/2022 00:00:00,M,Medico,2506,MICHELI MIKAELI COSTA DA PONTE SOUZA,Item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005641.0,CEFTRIAXONA 1G FA IV
16,9313035,3442388,498636,LAERCIO NOGUEIRA,03/01/2022 00:00:00,M,Medico,28663,RAISSA GABRIELLE REIS DOS SANTOS,Item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019360.0,PANTOZOL 40MG FA


In [ ]:
produtos_prescricao = ['ID_ITEM_PRESCRITO','ESQUEMA','DESCRICAO_DO_ESQUEMA','ITEM_DE_PRESCRICAO','NOME_ITEM_PRESCRICAO','FREQUENCIA','DESCRICAO_DA_FREQUENCIA','QUANTIDADE_PRESCRITA','DATA_HORA_ITEM','DIAS_PARA_APLICACAO','CODIGO_PROCEDIMENTO','DESCRICAO_PROCEDIMENTO', 'CD_PRODUTO','DS_PRODUTO']
cria_planilha(df_prescricao_produto[produtos_prescricao], 'produtos_prescricao.csv', dataset_final )

In [ ]:
sangue_prescricao = ['ID_ITEM_PRESCRITO','ESQUEMA','DESCRICAO_DO_ESQUEMA','ITEM_DE_PRESCRICAO','NOME_ITEM_PRESCRICAO','FREQUENCIA','DESCRICAO_DA_FREQUENCIA','QUANTIDADE_PRESCRITA','DATA_HORA_ITEM','DIAS_PARA_APLICACAO','CODIGO_PROCEDIMENTO','DESCRICAO_PROCEDIMENTO', 'CD_SANGUE_DERIVADOS', 'DS_SANGUE_DERIVADOS', 'CD_PRO_FAT_SANGUE', 'DS_PRO_FAT_SANGUE']
df_sangue_prescricao = df_prescricao[df_prescricao['CD_SANGUE_DERIVADOS'].isnull()==False][produtos_prescricao]
cria_planilha(df_sangue_prescricao[produtos_prescricao], 'sangue_prescricao.csv', dataset_final )

##Consumo

In [ ]:
df_consumo = le_planilha('consumo.csv', separador=";")

In [ ]:
df_consumo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178475 entries, 0 to 178474
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   CD_PRODUTO      178475 non-null  int64  
 1   DS_PRODUTO      178475 non-null  object 
 2   CD_ESPECIE      178475 non-null  int64  
 3   QT_CONSUMO      178475 non-null  float64
 4   CD_ATENDIMENTO  178475 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 6.8+ MB


In [ ]:
cria_planilha(df_consumo, 'consumo_atendimento.csv', dataset_final)